In [ ]:
def weighting_educ(column_mf, column_level):
    #Creating the observed data
    possible_levels = {"HS-grad", "Some-college", "Assoc-acdm", "assoc-voc", "Bachelors", "Masters", "Prof-school", "Doctorate"}
    p_obs_male_dict = {"8":0, "11":0, "HS-grad": 0, "Some-college": 0, "Assoc": 0, "Bachelors": 0, "Masters": 0, "Prof-school": 0, "Doctorate": 0}
    p_obs_female_dict = {"8":0, "11":0, "HS-grad": 0, "Some-college": 0, "Assoc": 0, "Bachelors": 0, "Masters": 0, "Prof-school": 0, "Doctorate": 0}
    female_ob = 0
    male_ob = 0
    for i,level,gender in enumerate(column_level, column_mf):
        #Counting all instances per category
        if not level in possible_levels:
            if int(level[0]) <= 8:
                if gender == "Male":
                    p_obs_male_dict["8"] += 1
                else:
                    p_obs_female_dict["8"] += 1
            else:
                if gender == "Male":
                    p_obs_male_dict["11"] += 1
                else:
                    p_obs_female_dict["11"] += 1  
        elif level in {"Assoc-acdm", "assoc-voc"}:
            if gender == "Male":
                    p_obs_male_dict["assoc"] += 1
            else:
                p_obs_female_dict["assoc"] += 1
        else:
            if gender == "Male":
                p_obs_male_dict[level] += 1
            else:
                p_obs_female_dict[level] += 1
        #Counting the amount of men and women
        if gender == "Male":
            male_ob += 1
        else:
            female_ob += 1
    #Changing the dicts used to count into lists
    p_obs_male = [p_obs_male_dict["8"], p_obs_male_dict["11"],p_obs_male_dict["HS-grad"],p_obs_male_dict["Some-college"],p_obs_male_dict["assoc"],p_obs_male_dict["Bachelors"], p_obs_male_dict["Masters"], p_obs_male_dict["Doctorate"]] / (male_ob + female_ob) * male_ob
    p_obs_female = [p_obs_female_dict["8"], p_obs_female_dict["11"],p_obs_female_dict["HS-grad"],p_obs_female_dict["Some-college"],p_obs_female_dict["assoc"],p_obs_female_dict["Bachelors"], p_obs_female_dict["Masters"], p_obs_female_dict["Doctorate"]] / (male_ob + female_ob) * female_ob
    
    #Calculating expected data using data from:
    #https://www.census.gov/data/tables/2022/demo/educational-attainment/cps-detailed-tables.html
    male = 109.979/226274
    female = 116.296/ 226274
    levels_male = [4041,	6303,	33081,	16085,	10616,	25192,	10156,	1860,	2644] /226274
    levels_female = [3895,	5695,	31383,	16900,	13057,	27853,	13725,	1584,	2203] /226274
    p_exp_male = []
    p_exp_female = []
    for i in range(len(levels_male)):
        p_exp_male.append(male * levels_male[i])
        p_exp_female.append(female * levels_female[i])
    
    #calculating the weights by deviding the expected by the observed. The output is in order of the dicts on the first lines of the function.
    weights_male = []
    weights_female = []
    for obs_m, exp_m, obs_f, exp_f in zip(p_obs_male, p_exp_male, p_obs_female, p_exp_female):
        weights_male.append(exp_m / obs_m)
        weights_female.append(exp_f / obs_f)
    
    return weights_male, weights_female
    